In [1]:
from pyspark.sql.functions import udf,col,split,regexp_extract_all,regexp_extract,explode,size,countDistinct, when, max, min, count
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

### Load dataset

In [2]:
# Create a Spark session
spark = SparkSession.builder.appName("REMDataAnalysis").master("local[*]") \
    .config("spark.driver.memory", "32g") \
    .config("spark.executor.meomory", "40g") \
        .getOrCreate()

    # .config("spark.executor.meomory", "24g") \
    #     .config("spark.driver.maxResultSize", "16g") \


24/01/29 21:07:16 WARN Utils: Your hostname, cssh-alpha resolves to a loopback address: 127.0.1.1; using 134.130.186.164 instead (on interface bond0)
24/01/29 21:07:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/29 21:07:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# spark.stop()

In [4]:
# Specify the path to the folder containing Orc files
input_folder_revision = "/home/cxu/swh-popular-3k-python/revision"
input_folder_snapshot = "/home/cxu/code-survival-analysis/snapshot_revisions_explored"
# input_folder_release = "/home/cxu/swh-popular-3k-python/release"
input_folder_origin_visit_status = "/home/cxu/swh-popular-3k-python/origin_visit_status"
# input_folder_origin_visit = "/home/cxu/swh-popular-3k-python/origin_visit"

# Read files into a DataFrame
revision = spark.read.format("orc").load(input_folder_revision)
snapshot = spark.read.format("json").load(input_folder_snapshot)
# release = spark.read.format("orc").load(input_folder_release)
origin_visit_status = spark.read.format("orc").load(input_folder_origin_visit_status)
# origin_visit = spark.read.format("orc").load(input_folder_origin_visit)

# Data description
# Print the schema of the dataset
tables = [revision, snapshot, origin_visit_status]
for data in tables:
    data.printSchema()
    # Show the first 5 rows of the dataset
    data.show(1)
    # Get the length of the dataset
    dataset_length = data.count()
    # Show the length of the dataset
    print(f"Length of the dataset: {dataset_length} rows")

root
 |-- id: string (nullable = true)
 |-- message: binary (nullable = true)
 |-- author: binary (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- date_offset: short (nullable = true)
 |-- committer: binary (nullable = true)
 |-- committer_date: timestamp (nullable = true)
 |-- committer_offset: short (nullable = true)
 |-- directory: string (nullable = true)

+--------------------+--------------------+--------------------+-------------------+-----------+--------------------+-------------------+----------------+--------------------+
|                  id|             message|              author|               date|date_offset|           committer|     committer_date|committer_offset|           directory|
+--------------------+--------------------+--------------------+-------------------+-----------+--------------------+-------------------+----------------+--------------------+
|e0b7070276027464e...|[66 69 78 69 6E 6...|[EE 2C E1 74 91 2...|2018-10-23 21:01:57|        120|

Length of the dataset: 76359551 rows
root
 |-- origin: string (nullable = true)
 |-- visit: long (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- status: string (nullable = true)
 |-- snapshot: string (nullable = true)

+--------------------+-----+--------------------+-------+--------------------+
|              origin|visit|                date| status|            snapshot|
+--------------------+-----+--------------------+-------+--------------------+
|deb://Debian/pack...|  287|2017-12-22 16:34:...|partial|48cfa88294968d0de...|
+--------------------+-----+--------------------+-------+--------------------+
only showing top 1 row

Length of the dataset: 784380 rows


In [5]:
snapshot = snapshot.dropDuplicates().na.drop()
origin_visit_status = origin_visit_status.dropDuplicates().na.drop()
revision = revision.dropDuplicates().na.drop()

In [6]:
# selected_columns = ["origin_visit_status.origin", "revision.author", "revision.date"]

In [7]:
result = revision \
    .join(snapshot, revision["id"] == snapshot["revision"])\
    .join(origin_visit_status, snapshot["snapshot"] == origin_visit_status["snapshot"])\
    .select(origin_visit_status["origin"], revision["id"], revision["author"], revision["date"])
 

In [8]:
result.show()

+--------------------+--------------------+--------------------+-------------------+
|              origin|                  id|              author|               date|
+--------------------+--------------------+--------------------+-------------------+
|https://gitlab.co...|000271b8d8efd7a00...|[BC DA FA F8 53 0...|2012-06-18 16:51:44|
|https://gitlab.co...|000271b8d8efd7a00...|[BC DA FA F8 53 0...|2012-06-18 16:51:44|
|https://gitlab.co...|000271b8d8efd7a00...|[BC DA FA F8 53 0...|2012-06-18 16:51:44|
|https://gitlab.co...|000271b8d8efd7a00...|[BC DA FA F8 53 0...|2012-06-18 16:51:44|
|https://gitlab.co...|000271b8d8efd7a00...|[BC DA FA F8 53 0...|2012-06-18 16:51:44|
|https://gitlab.co...|000271b8d8efd7a00...|[BC DA FA F8 53 0...|2012-06-18 16:51:44|
|https://github.co...|00033dbf403b08c9c...|[56 17 B0 21 3F B...|2007-08-18 22:05:58|
|https://github.co...|00033dbf403b08c9c...|[56 17 B0 21 3F B...|2007-08-18 22:05:58|
|https://github.co...|00033dbf403b08c9c...|[56 17 B0 21 3F B...|2

In [9]:
result = result.dropDuplicates().na.drop()

In [10]:
# result.count()

In [11]:
# result.select('author').distinct().count()

In [12]:
result.toPandas().to_csv('data/python3k_full.csv', index=False)

In [ ]:
spark.stop()